In [4]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone


In [9]:
icao = "EDDB" #Berlin Airport
date = datetime.now().date() #current date
time_1 = "00:00" #the time range for which I wanted to retrieve flight data
time_2 = "11:59"

url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{date}T{time_1}/{date}T{time_2}"

querystring = {"withLeg":"true",
               "direction":"Arrival",
               "withCancelled":"false",
               "withCodeshared":"true",
               "withCargo":"false",
               "withPrivate":"false"}

headers = {
    "X-RapidAPI-Key": 'Introduce your API Key',  # Replace with your actual API key
    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET",
                            url,
                            headers = headers,
                            params = querystring)

flights_json = response.json()

flights_json["arrivals"][0].keys()

dict_keys(['departure', 'arrival', 'number', 'callSign', 'status', 'codeshareStatus', 'isCargo', 'aircraft', 'airline'])

In [16]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone

def get_flight_data(icao_list):
    berlin_timezone = timezone('Europe/Berlin')
    today = datetime.now(berlin_timezone).date()
    tomorrow = (today + timedelta(days=1))

    flight_items = []

    for icao in icao_list:
        # the api can only make 12 hour calls, therefore, 2 12 hour calls make a full day
        # using the nested lists below we can make a morning call and extract the data
        # then make an afternoon call and extract the data
        times = [["00:00", "11:59"], ["12:00", "23:59"]]

        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

            querystring = {
                "withLeg": "true",
                "direction": "Arrival",
                "withCancelled": "false",
                "withCodeshared": "true",
                "withCargo": "false",
                "withPrivate": "false"
            }

            headers = {
                "X-RapidAPI-Key": 'Introduce your API key',  # Replace with your actual API key
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }

            response = requests.request("GET", url, headers=headers, params=querystring)
            flights_json = response.json()
            retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

            for item in flights_json["arrivals"]:
                flight_item = {
                    "arrival_airport_icao": icao,
                    "departure_airport_icao": item["departure"]["airport"].get("icao", None),
                    "scheduled_arrival_time": item["arrival"]["scheduledTime"].get("local", None),
                    "flight_number": item.get("number", None),
                    "date_retrieved_at": retrieval_time
                }

                flight_items.append(flight_item)

    flights_df = pd.DataFrame(flight_items)
    flights_df["scheduled_arrival_time"] = flights_df["scheduled_arrival_time"].str[:-6]
    flights_df["scheduled_arrival_time"] = pd.to_datetime(flights_df["scheduled_arrival_time"])
    flights_df["date_retrieved_at"] = pd.to_datetime(flights_df["date_retrieved_at"])

    return flights_df


In [17]:
# List of valid ICAO codes
icao_list = ["EDDM", "EDDH", "EDDB"]  # Example list

# Call the function and store the result in flights_df
flights_df = get_flight_data(icao_list)

flights_df.head()


,arrival_airport_icao,departure_airport_icao,scheduled_arrival_time,flight_number,date_retrieved_at
0,EDDM,VABB,2024-01-19 05:40:00,LH 767,2024-01-18 13:43:40
1,EDDM,WSSS,2024-01-19 05:45:00,LH 791,2024-01-18 13:43:40
2,EDDM,VTBS,2024-01-19 05:50:00,LH 773,2024-01-18 13:43:40
3,EDDM,ZBAA,2024-01-19 06:00:00,CA 961,2024-01-18 13:43:40
4,EDDM,LRTR,2024-01-19 06:05:00,LH 1661,2024-01-18 13:43:40


In [18]:
from sqlalchemy import create_engine

schema = "gans_db"  # name of the database you want to use here
host = "127.0.0.1"                 # to connect to your local server
user = "root"
password = "Introduce your password"      # your password!!!!
port = 3306
# Initialize engine
connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
engine = create_engine(connection_string)

flights_df.to_sql('flights_arrivals', con=engine, if_exists='append', index=False)


687